# TCC PUC - Ciência de Dados e Big Data

# PRÁTICA  DE ATIVIDADE FÍSICA ENTRE ADULTOS BRASILEIROS NAS CAPITAIS 

# 1. Definição do Problema

Descrever a prática de atividades fisica no tempo livre nas capitais do Brasil entre adultos brasileiros no periodo de 2016 a  2019. Será feita uma análise descritiva mais profunda no ano de 2019 com objetivo de encontrar atibutos com relevancia pra identificar e classificar quais tipo de atividade fisica e praticada tempo genero e idade.

Os dados serão tratados e trabalhados e posteriormente aplicaremos algoritmos Machine Learning para verificar quais atributos tem maior relevancia na pratica de atividade fisica. 

# Importando as bibliotecas

In [ ]:
#Importação da biblioteca pandas
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from datetime import date
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import seaborn as sb
from matplotlib import pyplot
import statsmodels.api as sm
import math

import operator

#Importação das funções para as medidas de avaliação dos algoritmos
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import warnings
warnings.filterwarnings('ignore')

# 2. Importando os Quatro arquivos XLS utilizando a função read_excel do tipo (parsing) para conversão dos arquivos em Dataframes

In [ ]:
##leitura dos 4 arquivos  XLS que contém os dados para os dataframes;
df_atividade_2016= pd.read_excel('Vigitel-2016-peso-rake.xls')
df_atividade_2017= pd.read_excel('Vigitel-2017-peso-rake.xls')
df_atividade_2018= pd.read_excel('Vigitel-2018-peso-rake.xls')
df_atividade_2019= pd.read_excel('Vigitel-2019-peso-rake.xls')


# 3. Concatenando todos os arquivos em um só

In [ ]:
# juntando os dataframes para um unico dataframe
df_atividade_junt = pd.merge(df_atividade_2016, df_atividade_2017, how = 'outer')

In [ ]:
# juntando os dataframes para um unico dataframe
df_atividade_junt2 = pd.merge(df_atividade_junt, df_atividade_2018, how = 'outer')

In [ ]:
# juntando os dataframes para um unico dataframe
df_atividade = pd.merge(df_atividade_junt2, df_atividade_2019, how = 'outer')

In [ ]:
#informações do dataframe df_atividade
df_atividade.info()

In [ ]:
#identificação das opções da coluna cidade do dataframe df_atividade
df_atividade['cidade'].unique()

In [ ]:
#Exibe as 1001 primeiras linhas
df_atividade.head(1001)

In [ ]:
#Informações do dataframe df_atividade
df_atividade.info()

# Verificando se existe campos nulos no Dataframe

In [ ]:
#Verificação dos valores nulos no dataframe df_atividade
df_atividade.isnull().sum()

# Tratando Campos nulos

In [ ]:
#trata valor nulo dataframe
df_atividade= df_atividade.drop(df_atividade.index[0])
df_atividade = df_atividade.fillna(0)

In [ ]:
# verifica o tratamento do nulo
df_atividade.isnull().sum()


# Exibe nomes das colunas Dataframe

In [ ]:
#Exibe todas as colunas do dataframe df_atividades
df_atividade.columns.values

# 3. Tratamento dos Dados

In [ ]:
# lista das colunas que vão ser usadas ColunasSelecionada 
ColunasSelecionada = ['ordem',  'ano', 'cidade', 'civil','q6', 'q7', 'q8a', 'q9', 'q11', 'q42', 'q43a', 'q44', 'q45','q46', 'q69']

In [ ]:
#Filtrar para o dataframe novo somente as colunas que foram selecionadas
df_atividadeSelecionadas = df_atividade.filter(items=ColunasSelecionada )

In [ ]:
#Exibe Dataframe df_atividadeSelecionadas

df_atividadeSelecionadas.head()


In [ ]:
#Renomeia as coluna 
df_atividadeSelecionadas.rename(columns={'q6':'idade','q7':'sexo','q8a':'grau_escolaridade','q9':'peso','q11':'altura','q42':'pratica_exercicio','q43a': 'tipo_exercicio','q44':'pratica_exercicio_1_vez_na_semana', 'q45':'frequencia_exercicio', 'q46':'duracao_exercicio', 'q69':'cor'}, inplace =True)

Cria coluna com o calculo IMC dataframe df_atividadeSelecionadas

In [ ]:
#criar coluna com o calculo IMC dataframe df_atividadeSelecionadas
df_atividadeSelecionadas['imc'] = df_atividadeSelecionadas.apply(
    lambda row: round(row.peso / (((row.altura * row.altura)/1000)*0.1),0), axis=1)
    

Dataframe df_censoSelecionadas2019Final vai ser usado no Treinamento

In [ ]:
df_censoSelecionadas2019FinalTreinamento = df_atividadeSelecionadas.loc[df_atividadeSelecionadas["ano"]== 2019]
df_censoSelecionadas2019Final = df_atividadeSelecionadas.loc[df_atividadeSelecionadas["ano"]== 2019]

In [ ]:
df_censoSelecionadas2019FinalTreinamento.head()

In [ ]:
#formate a coluna pratica exercicio 1 para sim e 2 para não 
df_atividadeSelecionadas['pratica_exercicio'].replace(1,'sim', inplace =True)
df_atividadeSelecionadas['pratica_exercicio'].replace(2,'nao', inplace =True)

#formate a coluna tipo exercicio
#1	caminhada (não vale deslocamento para trabalho)
#2	caminhada em esteira
#3	corrida (cooper)
#4	corrida em esteira
#5	musculação
#6	ginástica aeróbica (spinning, step, jump)
#7	hidroginástica
#8	ginástica em geral (alongamento, pilates, ioga)
#9	natação
#10	artes marciais e luta (jiu-jitsu, karatê, judô, boxe, muay thai, capoeira)
#11	bicicleta (inclui ergométrica)
#12	futebol/futsal
#13	basquetebol
#14	voleibol/futevolei
#15	tênis
#16	dança (balé, dança de salão, dança do ventre)
#17	outros
df_atividadeSelecionadas['tipo_exercicio'].replace(1,'caminhada (não vale deslocamento para trabalho)', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(2,'caminhada em esteira', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(3,'corrida (cooper)', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(4,'corrida em esteira', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(5,'musculação', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(6,'ginástica aeróbica (spinning, step, jump)', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(7,'hidroginástica', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(8,'ginástica em geral (alongamento, pilates, ioga)', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(9,'natação', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(10,'artes marciais e luta (jiu-jitsu, karatê, judô, boxe, muay thai, capoeira)', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(11,'bicicleta (inclui ergométrica)', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(12,'futebol/futsal', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(13,'basquetebol', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(14,'voleibol/futevolei', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(15,'tênis', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(16,'dança (balé, dança de salão, dança do ventre)', inplace =True)
df_atividadeSelecionadas['tipo_exercicio'].replace(17,'outros', inplace =True)

In [ ]:
#formate a coluna grau escolaridade
#1	curso primário
#2	admissão
#3	curso ginasial ou ginásio
#4	1º grau ou fundamental ou supletivo de 1º grau
#5	2º grau ou colégio ou técnico ou normal ou científico científico ou ensino médio ou supletivo de 2º grau
#6	3º grau ou curso superior
#7	pós-graduação (especialização, mestrado, doutorado)
#8	nunca estudou
#777	não sabe
#888	não quis responder

df_atividadeSelecionadas['grau_escolaridade'].replace(1,'curso primário', inplace =True)
df_atividadeSelecionadas['grau_escolaridade'].replace(2,'admissão', inplace =True)
df_atividadeSelecionadas['grau_escolaridade'].replace(3,'curso ginasial ou ginásio', inplace =True)
df_atividadeSelecionadas['grau_escolaridade'].replace(4,'1º grau ou fundamental ou supletivo de 1º grau', inplace =True)
df_atividadeSelecionadas['grau_escolaridade'].replace(5,'2º grau ou colégio ou técnico ou normal ou científico científico ou ensino médio ou supletivo de 2º grau', inplace =True)
df_atividadeSelecionadas['grau_escolaridade'].replace(6,'3º grau ou curso superior', inplace =True)
df_atividadeSelecionadas['grau_escolaridade'].replace(7,'pós-graduação (especialização, mestrado, doutorado)', inplace =True)
df_atividadeSelecionadas['grau_escolaridade'].replace(8,'nunca estudou', inplace =True)
df_atividadeSelecionadas['grau_escolaridade'].replace(777,'não sabe', inplace =True)
df_atividadeSelecionadas['grau_escolaridade'].replace(888,'não quis responder', inplace =True)

In [ ]:
#formate a coluna quantas vezes semana' 1 para sim e 2 para não
df_atividadeSelecionadas['pratica_exercicio_1_vez_na_semana'].replace(1,'sim', inplace =True)
df_atividadeSelecionadas['pratica_exercicio_1_vez_na_semana'].replace(2,'nao', inplace =True)

#formate a coluna Estado Civil

df_atividadeSelecionadas['civil'].replace(1,'solteiro', inplace =True)
df_atividadeSelecionadas['civil'].replace(2,'casado legalmente', inplace =True)
df_atividadeSelecionadas['civil'].replace(3,'tem união estável há mais de seis meses', inplace =True)
df_atividadeSelecionadas['civil'].replace(4,'viúvo', inplace =True)
df_atividadeSelecionadas['civil'].replace(5,'separado ou divorciado', inplace =True)
df_atividadeSelecionadas['civil'].replace(888,'não quis informar', inplace =True)


#formate a coluna frequencia exercicio
df_atividadeSelecionadas['frequencia_exercicio'].replace(1,'1 a 2 dias por semana', inplace =True)
df_atividadeSelecionadas['frequencia_exercicio'].replace(2,'3 a 4 dias por semana', inplace =True)
df_atividadeSelecionadas['frequencia_exercicio'].replace(3,'5 a 6 dias por semana', inplace =True)
df_atividadeSelecionadas['frequencia_exercicio'].replace(4,'todos os dias ( inclusive sábado e domingo)', inplace =True)


#formate a coluna duracao exercicio
df_atividadeSelecionadas['duracao_exercicio'].replace(1,'menos que 10 minutos', inplace =True)
df_atividadeSelecionadas['duracao_exercicio'].replace(2,'entre 10 e 19 minutos', inplace =True)
df_atividadeSelecionadas['duracao_exercicio'].replace(3,'entre 20 e 29 minutos', inplace =True)
df_atividadeSelecionadas['duracao_exercicio'].replace(4,'entre 30 e 39 minutos', inplace =True)
df_atividadeSelecionadas['duracao_exercicio'].replace(5,'entre 40 e 49 minutos', inplace =True)
df_atividadeSelecionadas['duracao_exercicio'].replace(6,'entre 50 e 59 minutos', inplace =True)
df_atividadeSelecionadas['duracao_exercicio'].replace(7,'60 minutos ou mais', inplace =True)


#formate a coluna cor
df_atividadeSelecionadas['cor'].replace(1,'branca', inplace =True)
df_atividadeSelecionadas['cor'].replace(2,'preta', inplace =True)
df_atividadeSelecionadas['cor'].replace(3,'amarela', inplace =True)
df_atividadeSelecionadas['cor'].replace(4,'parda', inplace =True)
df_atividadeSelecionadas['cor'].replace(5,'indígena', inplace =True)
df_atividadeSelecionadas['cor'].replace(80,'Morena', inplace =True)
df_atividadeSelecionadas['cor'].replace(777,'não sabe', inplace =True)
df_atividadeSelecionadas['cor'].replace(888,'não quis informar', inplace =True)


In [ ]:
#formate a coluna quantas vezes semana' 1 para masculino e 2 para feminino
df_atividadeSelecionadas['sexo'].replace(1,'masculino', inplace =True)
df_atividadeSelecionadas['sexo'].replace(2,'feminino', inplace =True)

# Visualização dos Tipos de cada Coluna

In [ ]:
#Verifica o tipo do dados no Dataframe
df_atividadeSelecionadas.info()

# Ajustando os Dados para os Formatos Corretos e Otimização dos Tipos de dados para economizar memoria

In [ ]:
#Ajustando os Dados para os Formatos Corretos e Otimização dos Tipos de dados para economizar memoria

df_atividadeSelecionadas.ordem   = df_atividadeSelecionadas.ordem.astype('int32')
df_atividadeSelecionadas.ano     = df_atividadeSelecionadas.ano.astype('int32')
df_atividadeSelecionadas.cidade  = df_atividadeSelecionadas.cidade.astype('int32')
df_atividadeSelecionadas.civil  = df_atividadeSelecionadas.civil.astype('category')
df_atividadeSelecionadas.idade  = df_atividadeSelecionadas.idade.astype('int32')
df_atividadeSelecionadas.sexo   = df_atividadeSelecionadas.sexo.astype('category')
df_atividadeSelecionadas.grau_escolaridade  = df_atividadeSelecionadas.grau_escolaridade.astype('category')
df_atividadeSelecionadas.peso   = df_atividadeSelecionadas.peso.astype('float64')
df_atividadeSelecionadas.altura = df_atividadeSelecionadas.altura.astype('int32')
df_atividadeSelecionadas.pratica_exercicio  = df_atividadeSelecionadas.pratica_exercicio.astype('category')
df_atividadeSelecionadas.tipo_exercicio     = df_atividadeSelecionadas.tipo_exercicio.astype('category')
df_atividadeSelecionadas.pratica_exercicio_1_vez_na_semana = df_atividadeSelecionadas.pratica_exercicio_1_vez_na_semana.astype('category')
df_atividadeSelecionadas.frequencia_exercicio = df_atividadeSelecionadas.frequencia_exercicio.astype('category')
df_atividadeSelecionadas.duracao_exercicio = df_atividadeSelecionadas.duracao_exercicio.astype('category')
df_atividadeSelecionadas.cor = df_atividadeSelecionadas.cor.astype('category')



In [ ]:
# Exibe os campos tratados
df_atividadeSelecionadas.head()

In [ ]:
#Visualização de Outliers
sns.boxplot(x=df_atividadeSelecionadas['altura'])


# Obtém o Segundo Dataframe para ser usado com informações do ANO 2019

In [ ]:
#Tratar dados da Tabela Censo ano 2019 que vai ser usada para extrair informacões da Cidade como renda per capita, nome estado, Regiao etc...
df_censo= pd.read_excel('Capitais_Censo.xls')

In [ ]:
#exibe dados do segundo Dataframe df_censo demogradico
df_censo.head()

In [ ]:
#Mostrar todas as colunas do dataframe df_censo demografico
df_censo.columns.values

# Trata os dados do Dataset ColunasSelecionadaCenso

In [ ]:
#Criar lista das colunas que vão ser usadas Censo
ColunasSelecionadaCenso = ['cidade', 'uf', 'capital', 'populacao estimada - pessoas [2021]', 'pib per capita', 'regiao', 'ano','Salario_medio_mensal']

In [ ]:
#Filtra para o dataframe novo somente as colunas que foram selecionadas
df_censoSelecionadas = df_censo.filter(items=ColunasSelecionadaCenso )

In [ ]:
# Exibe linhas do Dataset
df_censoSelecionadas.head()

In [ ]:
#Renomeia as coluna 
df_censoSelecionadas.rename(columns={'capital':'nome_capital', 'populacao estimada - pessoas [2021]': 'populacao_estimada_pessoas','pib per capita': 'pib_per_capita'}, inplace =True)


In [ ]:
df_censoSelecionadas.head()

In [ ]:
#criar coluna com o calculo Salario dataframe df_censoSelecionadas
df_censoSelecionadas['Salario'] = df_censoSelecionadas.apply(
    lambda row: round ((row.Salario_medio_mensal * 998.00),0), axis=1)

In [ ]:
#criar coluna com o calculo PIB dataframe df_censoSelecionadas
df_censoSelecionadas['PIB'] = df_censoSelecionadas.apply(
    lambda row: round ((row.pib_per_capita * row.populacao_estimada_pessoas),0), axis=1)

In [ ]:
#formata campo para flutuante

pd.set_option('float_format', '{:.2f}'.format)

In [ ]:
# Exibe linhas do Datase
df_censoSelecionadas.head()

In [ ]:
df_censoSelecionadas.info()

# Concatenando Dataframe 2019 em um só

In [ ]:
##Unindo os datasets df_censoSelecionadas com df_atividadeSelecionada2019
###
###
df_atividadeSelecionada2019 = df_atividadeSelecionadas.loc[df_atividadeSelecionadas["ano"]== 2019]

df_censoSelecionadas2019 = pd.merge(df_atividadeSelecionada2019, df_censoSelecionadas, how = 'outer')

df_censoSelecionadas2019Final = pd.merge(df_censoSelecionadas2019Final, df_censoSelecionadas, how = 'outer')


In [ ]:
# Exibe linhas do Dataset
df_atividadeSelecionada2019.head()

# 4. Análise e Exploração dos Dados 

Exibindo a dimensão do DataFrame. Logo percebe-se que teremos 211081 linhas e 16 colunas.

In [ ]:
df_atividadeSelecionadas.shape

In [ ]:
#verifica o total se pratica ou não Atividades Fisicas
pequisa = Counter(df_atividadeSelecionadas['pratica_exercicio'])
pequisa 

Exibe o valor total dos entrevistados por Ano

In [ ]:
#Plotagem das informações numero Total de pesquisa ANO
plt.figure(figsize=(20,4))
plt.subplot(131)
sns.countplot(x= 'ano', data = df_atividadeSelecionadas, palette="GnBu_d",edgecolor="black")
plt.title("Total Entrevistados ANO", fontsize=18)

plt.show()

In [ ]:
#Plotagem das informações de gênero dos Entrevistados

import plotly.express as px
pie = df_atividadeSelecionadas['pratica_exercicio'].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_atividadeSelecionadas, values=population, names=regions, title ="Total de entrevistados para os ANOS 2016, 2017, 2018, 2019 Agrupado por Ano")
fig.show()

Plotagem das informações de gênero dos Entrevistados Total

In [ ]:
#Plotagem das informações de gênero dos Entrevistados

import plotly.express as px
pie = df_atividadeSelecionadas["sexo"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_atividadeSelecionadas, values=population, names=regions, title ="Total de entrevistados para os ANOS 2016, 2017, 2018, 2019 Agrupado por Gênero")
fig.show()

Tratar dataframes para dois dataframes SIM e Não para a pratica de atividade fisica.

In [ ]:
#Plotagem das informações de gênero dos Entrevistados

import plotly.express as px
pie = df_atividadeSelecionadas.loc[df_atividadeSelecionadas["pratica_exercicio"]== "sim"]
pie = pie["sexo"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_atividadeSelecionadas, values=population, names=regions, title ="Total de entrevistados para os ANOS 2016, 2017, 2018, 2019 Agrupados por Gênero")
fig.show()

In [ ]:
#filtra atividade fisica e colocar o resultado no dataframe dftotalpraticaSim
dftotalpraticaSim= df_atividadeSelecionadas.loc[df_atividadeSelecionadas["pratica_exercicio"]== "sim"]


#filtra não pratica atividade fisica e colocar o resultado no dataframe dftotalpraticaNao
dftotalpraticaNao= df_atividadeSelecionadas.loc[df_atividadeSelecionadas["pratica_exercicio"]== "nao"]
#verifica o total de 
pequisa = Counter(df_atividadeSelecionadas['ano'])
pequisa 

In [ ]:
# conta quantos registros tem o dataframe pequisaSim
pequisaSim = Counter(dftotalpraticaSim['ano'])
pequisaSim


Exibe as informações dos Entrevistados que praticam atividade fisica porcentagem

In [ ]:
#Plotagem das informaçõesdos Entrevistados que praticam atividade fisica porcentagem
plt.style.use('seaborn-pastel')
plt.pie(pequisaSim.values(),labels = pequisaSim.keys(),
       autopct = '%1.1f%%', textprops={'fontsize':16})
plt.axis("image")
plt.title("Porcentagem de entrevistados que respoderam SIM", fontsize=18)
plt.show

In [ ]:
pequisaNao = Counter(dftotalpraticaNao['ano'])
pequisaNao

Exibe as informações dos Entrevistados total evolução ano gênero Pratica de Atividade Fisica

In [ ]:
#Plotagem das informações dos Entrevistados que não praticam atividade fisica porcentagem
plt.style.use('seaborn-pastel')
plt.pie(pequisaNao.values(),labels = pequisaNao.keys(),
       autopct = '%1.1f%%', textprops={'fontsize':16})
plt.axis("image")
plt.title("Porcentagem de entrevistados que respoderam Não", fontsize=18)
plt.show

In [ ]:
#Plotagem das informações dos Entrevistados total evolução ano
df1=df_atividadeSelecionadas[['pratica_exercicio','ano']]
df2=df1.groupby(['ano','pratica_exercicio']).size().reset_index(name='Total Entrevistados')
fig3 = px.line(df2, x="ano", y="Total Entrevistados", color='pratica_exercicio',title='Evolução Pratica de atividade Fisica ao Longo dos Anos')
fig3.show()

In [ ]:
#Plotagem das informações de gênero dos Entrevistados Praticam Atividades Fisicas

import plotly.express as px
pie = dftotalpraticaSim['sexo'].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(dftotalpraticaSim, values=population, names=regions, title ="Total de entrevistados por gênero que praticam Atividades Fisicas")
fig.show()
#verifica o total entrevitas agrupado por ano
pequisa1 = Counter(dftotalpraticaSim['sexo'])
pequisa1

Exibe as informações de gênero dos Entrevistados que não Praticam Atividades Fisicas

In [ ]:
#Plotagem das informações de gênero dos Entrevistados que não Praticam Atividades Fisicas

import plotly.express as px
pie = dftotalpraticaNao["sexo"].value_counts()
regions = pie.index

population = pie.values
fig = px.pie(dftotalpraticaNao, values=population, names=regions, title ="Total de entrevistados por gênero que não praticam Atividades Fisicas")
fig.show()

In [ ]:
#Plotagem das informações de gênero dos Entrevistados Total
import plotly.express as px
df = df_atividadeSelecionadas
figure = px.histogram(df, x = "sexo", color = "pratica_exercicio", title= "Pesquisa Total ANO")
figure.show()

In [ ]:
# conta quantos registros tem o dataframe pequisaNao
pequisaNao = Counter(dftotalpraticaNao['ano'])
pequisaNao

In [ ]:
#Contagem de modalidade por numero de praticantes
PraticaAtivida = Counter(dftotalpraticaSim['tipo_exercicio'])
PraticaAtivida

Informações dos Entrevistados que não praticam atividade fisica porcentagem

Exibe as informações dos Tipos de Atividade Fisica total agrupado por modalidade

In [ ]:
 #Plotagem das informações dos Entrevistados total evolução ano
df= dftotalpraticaSim
import plotly.express as px
pie = df["tipo_exercicio"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df, values=population, names=regions, title ="Total de entrevistados para os ANOS 2016, 2017, 2018, 2019 separados por Modalidade")
fig.show()   
    

Exibe o IMC maior que 15 e menor que 50 para todos os entrevistados

In [ ]:
#Plotagem das informações numero Total de pesquisa ANO

df1=df_atividadeSelecionadas[['imc','ano']]
df1= df1.loc[(df1['imc']> 15 ) & (df1['imc']< 50)]
df2=df1.groupby(['ano','imc']).size().reset_index(name='Total Entrevistados')
fig3 = px.line(df2, x="ano", y="Total Entrevistados", color='imc',title='Evolução IMC ao Longo dos Anos')
fig3.show()


Exibe histograma com as idade dos Praticante de Atividade Fisica

In [ ]:
#Plotagem de histograma com as idade Praticante Atividade Fisica
dftotalpraticaSim.idade.hist(bins=20)
plt.style.use('seaborn-pastel')
plt.xlabel("idade")
plt.ylabel("Número de Entrevistado que pratica Atividade Fisica")
plt.title("Idade Praticante Atividade Fisica")
plt.show()

In [ ]:
#Plotagem de histograma com as idade Praticante Atividade Fisica
dftotalpraticaNao.idade.hist(bins=20)
plt.style.use('seaborn-pastel')
plt.xlabel("idade")
plt.ylabel("Número de Entrevistado que Não pratica Atividade Fisica")
plt.title("Idade Praticante Atividade Fisica")
plt.show()

In [ ]:
# Media das idades dos praticantes de atividades fisica 
dftotalpraticaSim['idade'].mean()

# Vamos analisar um período especifico ANO 2019

Exibe total de entrevistados para os ANOS 2019 separados por Gênero

In [ ]:
# conta quantos registros tem o dataframe df_censoSelecionadas2019
pequisa2019 = Counter(df_censoSelecionadas2019['ano'])
pequisa2019

In [ ]:
# conta quantos registros tem o dataframe df_censoSelecionadas2019
pequisa2019Pra = Counter(df_censoSelecionadas2019['pratica_exercicio'])
pequisa2019Pra

In [ ]:
#Vamos tratar um perriodo Especifico ANO 2019 

#Plotagem das informações de gênero dos Entrevistados

#filtra atividade fisica e colocar o resultado no dataframe dftotalpraticaSim
df_censoSelecionadas2019Sim = df_censoSelecionadas2019.loc[df_censoSelecionadas2019['pratica_exercicio']== 'sim']
df_censoSelecionadas2019Nao = df_censoSelecionadas2019.loc[df_censoSelecionadas2019['pratica_exercicio']== 'nao']
df_censoSelecionadas2019teste = df_censoSelecionadas2019
##df_censoSelecionadas2019.head()

import plotly.express as px
pie = df_censoSelecionadas2019["pratica_exercicio"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_censoSelecionadas2019, values=population, names=regions, title ="Total de entrevistados para os ANOS 2019 separados por Sim ou Não")
fig.show()

In [ ]:
#Vamos tratar um perriodo Especifico ANO 2019 

#Plotagem das informações de gênero dos Entrevistados

#filtra atividade fisica e colocar o resultado no dataframe dftotalpraticaSim
df_censoSelecionadas2019Sim = df_censoSelecionadas2019.loc[df_censoSelecionadas2019['pratica_exercicio']== 'sim']
df_censoSelecionadas2019Nao = df_censoSelecionadas2019.loc[df_censoSelecionadas2019['pratica_exercicio']== 'nao']
df_censoSelecionadas2019teste = df_censoSelecionadas2019
##df_censoSelecionadas2019.head()

import plotly.express as px
pie = df_censoSelecionadas2019["sexo"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_censoSelecionadas2019, values=population, names=regions, title ="Total de entrevistados para os ANOS 2019 separados por Gênero")
fig.show()

In [ ]:
###Plotagem das informações Tipo Atividade Fisica dos Entrevistados ("Pratica(SIM,NÃO) e Sexo")
sns.countplot(df_censoSelecionadas2019.pratica_exercicio, hue=df_censoSelecionadas2019.sexo )
plt.title("Pratica(SIM,NÃO) e Sexo")
plt.show()


Exibe as informações de gênero dos Entrevistados que pratica atividade fisica porcentagem


In [ ]:
##Plotagem das informações de gênero dos Entrevistados que pratica atividade fisica porcentagem

import plotly.express as px
pie = df_censoSelecionadas2019Sim["sexo"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_censoSelecionadas2019Sim, values=population, names=regions, title ="Porcentagem de entrevistados que respoderam SIM 2019")
fig.show()

In [ ]:
#Informações do campo idade
df_censoSelecionadas2019['idade'].describe()

In [ ]:
#Informações do campo IMC
df_censoSelecionadas2019['imc'].describe()

Exibe histograma com todas as campos do dataframe df_censoSelecionadas2019Sim


In [ ]:
#Plotagem de histograma com as idade Entrevistados
df_censoSelecionadas2019.idade.hist(bins=20)
plt.style.use('seaborn-pastel')
plt.xlabel("idade")
plt.ylabel("Número de Entrevistados")
plt.title("Idade Entrevistados")
plt.show()

In [ ]:
#Informações do campo idade Praticande de Atividade
df_censoSelecionadas2019Sim['idade'].describe()

In [ ]:
#Informações do campo idade Não Praticande de Atividade
df_censoSelecionadas2019Nao['idade'].describe()

In [ ]:
#Plotagem de histograma com as idade Não Praticante Atividade Fisica
df_censoSelecionadas2019Nao.idade.hist(bins=20)
plt.style.use('seaborn-pastel')
plt.xlabel("idade")
plt.ylabel("Número de Entrevistado que pratica Atividade Fisica")
plt.title("Idade Praticante Atividade Fisica")
plt.show()

In [ ]:
#Plotagem de histograma com as idade Praticante Atividade Fisica
df_censoSelecionadas2019Sim.idade.hist(bins=20)
plt.style.use('seaborn-pastel')
plt.xlabel("idade")
plt.ylabel("Número de Entrevistado que pratica Atividade Fisica")
plt.title("Idade Praticante Atividade Fisica")
plt.show()

Exibe evolução Pratica de atividade Fisica ao Longo da Idade

In [ ]:
#Plotagem das informações dos Entrevistados total evolução ano 2019
df1=df_censoSelecionadas2019[['pratica_exercicio','idade']]
df2=df1.groupby(['idade','pratica_exercicio']).size().reset_index(name='Total Entrevistados')
fig3 = px.line(df2, x="idade", y="Total Entrevistados", color='pratica_exercicio',title='Evolução Pratica de atividade Fisica ou não ao Longo da Idade')
fig3.show()

In [ ]:
#filtra atividade fisica e colocar o resultado no dataframe dftotalpraticaSim
df_censoSelecionadas2019Sim= df_censoSelecionadas2019.loc[df_censoSelecionadas2019["pratica_exercicio"]== "sim"]


#filtra não pratica atividade fisica e colocar o resultado no dataframe dftotalpraticaNao
df_censoSelecionadas2019Nao= df_censoSelecionadas2019.loc[df_censoSelecionadas2019["pratica_exercicio"]== "nao"]
#verifica o total de 
df_pequisa2019 = Counter(df_censoSelecionadas2019['ano'])
df_pequisa2019 

Total Pequisa Sim 2019

In [ ]:
df_pequisa2019SIM = Counter(df_censoSelecionadas2019Sim['ano'])
df_pequisa2019SIM 

Total Pequisa Não 2019

In [ ]:
df_pequisa2019NAO = Counter(df_censoSelecionadas2019Nao['ano'])
df_pequisa2019NAO

In [ ]:
#Descrição estatística da idade dos Entrevistados Sim 2019
df_censoSelecionadas2019Sim['idade'].describe()

In [ ]:
#Nome colunas daaframe df_censoSelecionadas2019Sim
df_censoSelecionadas2019Sim.columns.values

In [ ]:
#exibe as cor dos entrevistados SIM
df_censoSelecionadas2019Sim["cor"].value_counts()

In [ ]:
#exibe as cor dos entrevistados SIM
df_censoSelecionadas2019Nao["cor"].value_counts()

In [ ]:
##Plotagem das informações Tipo Atividade Fisica dos Entrevistados por COR porcentagem
import plotly.express as px
pie =df_censoSelecionadas2019["cor"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_censoSelecionadas2019, values=population, names=regions, title ="Porcentagem de entrevistados Por Cor")
fig.show()

# Exibe as informações Tipo Atividade Fisica dos Entrevistados por COR porcentagem

In [ ]:
##Plotagem das informações Tipo Atividade Fisica dos Entrevistados por COR porcentagem
import plotly.express as px
pie =df_censoSelecionadas2019Sim["cor"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_censoSelecionadas2019Sim, values=population, names=regions, title ="Porcentagem de entrevistados por Cor praticante de atividade")
fig.show()

Exibe as informações Tipo Atividade Fisica dos Entrevistados por COR e Gênero

In [ ]:
###Plotagem das informações Tipo Atividade Fisica dos Entrevistados cor da pele e sexo
sns.countplot(df_censoSelecionadas2019Sim.cor, hue=df_censoSelecionadas2019Sim.sexo )
plt.title("Cor e Sexo")
plt.show()


Exibe as informações Atividade Fisica dos Entrevistados por Região

In [ ]:
##Plotagem das informações Atividade Fisica dos Entrevistados que praticam atividade porcentagem por Região

import plotly.express as px
pie = df_censoSelecionadas2019Sim["regiao"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_censoSelecionadas2019Sim, values=population, names=regions, title ="Porcentagem de entrevistados Por Regiao")
fig.show()

In [ ]:
#Atividade Fisica dos Entrevistados que praticam atividade porcentagem por Região

df_censoSelecionadas2019Sim["regiao"].value_counts()

In [ ]:
##Plotagem das informações Atividade Fisica dos Entrevistados que Não praticam atividade porcentagem por Estado Civil

import plotly.express as px
pie = df_censoSelecionadas2019Nao["civil"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_censoSelecionadas2019Nao, values=population, names=regions, title ="Porcentagem de entrevistados Por Estado Civil")
fig.show()

In [ ]:
df_casados =df_censoSelecionadas2019Sim.loc[df_censoSelecionadas2019Sim['civil']== 'casado legalmente']
df_casados['idade'].describe()

In [ ]:

df_censoSelecionadas2019Sim['idade'].describe()

In [ ]:
#exibe lista estado Civil
df_pequisa2019SIMEstadoCivil = Counter(df_censoSelecionadas2019Sim["civil"])
df_pequisa2019SIMEstadoCivil


In [ ]:
##Plotagem das informações Atividade Fisica dos Entrevistados que praticam atividade porcentagem por Estado Civil

import plotly.express as px
pie = df_censoSelecionadas2019Sim["civil"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_censoSelecionadas2019Sim, values=population, names=regions, title ="Porcentagem de entrevistados Por Estado CIVIL")
fig.show()

Exibe as informações Atividade Fisica dos Entrevistados que praticam atividade porcentagem agrupado por Cidade

In [ ]:
##Plotagem das informações Tipo Atividade Fisica dos Entrevistados que praticam atividade porcentagem agrupado por Cidade

import plotly.express as px
pie = df_censoSelecionadas2019Sim["nome_capital"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_censoSelecionadas2019Sim, values=population, names=regions, title ="Porcentagem de entrevistados que respoderam SIM 2019 agrupado por Cidade")
fig.show()

In [ ]:
# Conta quantos Respoderam sim por Capital
df_censoSelecionadas2019Sim["nome_capital"].value_counts()

In [ ]:
df_censoSelecionadas2019Sim.head()

In [ ]:
#Plotagem das informações dos Entrevistados total evolução ano 2019
df1=df_censoSelecionadas2019Sim[['nome_capital','imc',]]
df1=df1.loc[(df1['imc']> 15 ) & (df1['imc']< 50)]
df2=df1.groupby(['imc','nome_capital']).size().reset_index(name='Total Entrevistados')
fig3 = px.line(df2, x="imc", y="Total Entrevistados", color='nome_capital',title='Evolução Pratica de atividade Fisica ao Longo do IMC')
fig3.show()

Exibe top 10 Capitais na Pratica de exercicios

In [ ]:
import matplotlib as mpl
game = df_censoSelecionadas2019Sim.groupby("nome_capital")["ano"].count().head(10)
custom_colors = mpl.colors.Normalize(vmin=min(game), vmax=max(game))
colours = [mpl.cm.PuBu(custom_colors(i)) for i in game]
plt.figure(figsize=(7,7))
plt.pie(game, labels=game.index, colors=colours)
central_circle = plt.Circle((0, 0), 0.5, color='white')
fig = plt.gcf()
fig.gca().add_artist(central_circle)
plt.rc('font', size=13)
plt.title("Top 10 Capitais Pratica de exercicio ", fontsize=20)
plt.show()

In [ ]:
#seleciona coluna Nome Capital e salario

ColunasSelecionadaCenso = ['nome_capital','Salario']
SalarioCapital = df_censoSelecionadas.filter(items=ColunasSelecionadaCenso)
SalarioCapital

In [ ]:
#Plotagem do gráfico Salario por capital
plt.style.use('seaborn-pastel')
plt.barh( df_censoSelecionadas.nome_capital, df_censoSelecionadas.Salario,)
plt.ylabel('Salario')
plt.xlabel('nome_capital')
plt.title('Salario Medio Por capital')
plt.show()

In [ ]:
##Plotagem das informações Salario porcentagem agrupado por Cidade

import plotly.express as px
pie = df_censoSelecionadas["Salario"]
regions =df_censoSelecionadas["nome_capital"] 
population = pie.values
fig = px.pie(df_censoSelecionadas, values=population, names=regions, title ="Porcentagem Salario agrupado por salario")
fig.show()

In [ ]:
#seleciona coluna Nome Capital e PIB

ColunasSelecionadaCenso = ['nome_capital','PIB']
SalarioCapital = df_censoSelecionadas.filter(items=ColunasSelecionadaCenso)
SalarioCapital

In [ ]:
##Plotagem das informações PIB porcentagem agrupado por Cidade

import plotly.express as px
pie = SalarioCapital["PIB"]
regions =SalarioCapital["nome_capital"] 
population = pie.values
fig = px.pie(df_censoSelecionadas, values=population, names=regions, title ="Porcentagem Salario agrupado por PIB")
fig.show()

In [ ]:
 ##Plotagem das informações Total população porcentagem agrupado por Cidade

ColunasSelecionadaCenso = ['nome_capital','populacao_estimada_pessoas']
SalarioCapital = df_censoSelecionadas.filter(items=ColunasSelecionadaCenso)
SalarioCapital

In [ ]:
#Total População
dfCont= sum(SalarioCapital.populacao_estimada_pessoas)
dfCont

In [ ]:
 ##Plotagem das informações Total população porcentagem agrupado por Cidade

import plotly.express as px
pie = df_censoSelecionadas["Salario"]
regions =df_censoSelecionadas["nome_capital"] 
population = pie.values
fig = px.pie(df_censoSelecionadas, values=population, names=regions, title ="Porcentagem Contagem população agrupado por cidade")
fig.show()

In [ ]:
#Plotagem do gráfico Salario por capital

plt.style.use('seaborn-pastel')
plt.barh( df_censoSelecionadas.nome_capital, df_censoSelecionadas.Salario,)
plt.ylabel('PIB')
plt.xlabel('nome_capital')
plt.title('Valor PIB')
plt.show()

In [ ]:
#Plotagem das informações Duração dos Exercicios por Frequencia Exercicio
import plotly.express as px
df = df_censoSelecionadas2019Sim
figure = px.histogram(df, x = "tipo_exercicio", color = "frequencia_exercicio", title= "Duração dos Exercicios por Frquencia Agrupado Exercicio")
figure.show()

In [ ]:
#Plotagem das informações Duração dos Exercicios por Capital"
import plotly.express as px
df = df_censoSelecionadas2019Sim 
figure = px.histogram(df, x = "nome_capital", color = "tipo_exercicio", title= "Tipo de Exercicios por Capital")
figure.show()

In [ ]:
#Plotagem das informações Duração dos Exercicios por Frequencia por Cor
import plotly.express as px
df = df_censoSelecionadas2019Sim
figure = px.histogram(df, x = "cor", color = "frequencia_exercicio", title= "Duração dos Exercicios por Frquencia Agrupado Por Cor")
figure.show()

In [ ]:

#Plotagem das informações Duração dos Exercicios por Tipo Exercicio"
import plotly.express as px
df = df_censoSelecionadas2019Sim
figure = px.histogram(df, x = "nome_capital", color = "duracao_exercicio", title= "Duração dos Exercicios por Tipo Exercicio")
figure.show()

In [ ]:
# Total agrupado por atividade fisica
df_censoSelecionadas2019Sim["tipo_exercicio"].value_counts()

In [ ]:
##Plotagem das informações Tipo Atividade Fisica dos Entrevistados que pratica atividade porcentagem agrupado por Atividade

import plotly.express as px
pie = df_censoSelecionadas2019Sim["tipo_exercicio"].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df_censoSelecionadas2019Sim, values=population, names=regions, title ="Porcentagem de entrevistados que respoderam SIM 2019 agrupado por Atividade")
fig.show()

In [ ]:
df_censoSelecionadas.head()

In [ ]:
#Plotagem das informações dos Entrevistados total evolução ano 2019
df1=df_censoSelecionadas2019Sim[['tipo_exercicio','idade',]]
df2=df1.groupby(['idade','tipo_exercicio']).size().reset_index(name='Total Entrevistados')
fig3 = px.line(df2, x="idade", y="Total Entrevistados", color='tipo_exercicio',title='Evolução Pratica de atividade Fisica ao Longo da Idade por Tipo Exercicio')
fig3.show()

In [ ]:
#Plotagem das informações Duração dos Exercicios por COR"
import plotly.express as px
df = df_censoSelecionadas2019Sim
figure = px.histogram(df, x = "cor", color = "tipo_exercicio", title= "Tipo de Exercicios por COR")
figure.show()

In [ ]:
#Plotagem das informações Duração dos Exercicios por COR"
import plotly.express as px
df = df_censoSelecionadas2019Sim
figure = px.histogram(df, x = "cor", color = "duracao_exercicio", barmode ='group', title= "Duração dos Exercicios por COR")
figure.show()

In [ ]:

#Plotagem das informações Duração dos Exercicios por Tipo Exercicio"
import plotly.express as px
df = df_censoSelecionadas2019Sim
figure = px.histogram(df, x = "tipo_exercicio", color = "duracao_exercicio", title= "Duração dos Exercicios por Tipo Exercicio")
figure.show()

Exibe Top 10 Tipo Exercico praticados

In [ ]:
import matplotlib as mpl
game = df_censoSelecionadas2019Sim.groupby("tipo_exercicio")["ano"].count().head(10)
custom_colors = mpl.colors.Normalize(vmin=min(game), vmax=max(game))
colours = [mpl.cm.PuBu(custom_colors(i)) for i in game]
plt.figure(figsize=(7,7))
plt.pie(game, labels=game.index, colors=colours)
central_circle = plt.Circle((0, 0), 0.5, color='white')
fig = plt.gcf()
fig.gca().add_artist(central_circle)
plt.rc('font', size=12)
plt.title("Top 10 Tipo Exercico praticado ", fontsize=20)
plt.show()

In [ ]:
#Conta lista exercicos
df_censoSelecionadas2019Sim["tipo_exercicio"].value_counts()

Evolução Pratica de atividade Fisica ao Longo da Idade por Cidade

In [ ]:
#Plotagem das informações dos Entrevistados total evolução ano 2019
df1=df_censoSelecionadas2019[['nome_capital','idade']]
df2=df1.groupby(['idade','nome_capital']).size().reset_index(name='Total Entrevistados')
fig3 = px.line(df2, x="idade", y="Total Entrevistados", color='nome_capital',title='Evolução Pratica de atividade Fisica ao Longo da Idade por Cidade')
fig3.show()

Evolução Pratica de atividade Fisica ao Longo da Idade por Região

In [ ]:
#Plotagem das informações dos Entrevistados total evolução Região
df1=df_censoSelecionadas2019Sim[['regiao','idade']]
df2=df1.groupby(['idade','regiao']).size().reset_index(name='Total Entrevistados')
fig3 = px.line(df2, x="idade", y="Total Entrevistados", color='regiao',title='Evolução Pratica de atividade Fisica ao Longo da Idade por Região')
fig3.show()

Evolução Pratica de atividade Fisica ao Longo da Idade por Cor

In [ ]:
#Plotagem das informações dos Entrevistados total evolução ano 2019
df1=df_censoSelecionadas2019Sim[['cor','idade',]]
df2=df1.groupby(['idade','cor']).size().reset_index(name='Total Entrevistados')
fig3 = px.line(df2, x="idade", y="Total Entrevistados", color='cor',title='Evolução Pratica de atividade Fisica ao Longo da Idade por Cor')
fig3.show()

Exibe Evolução Pratica de atividade Fisica ao Longo da Idade por Cidades região Sudeste

In [ ]:
#Plotagem das informações dos Entrevistados total evolução ano 2019
df_cidade = df_censoSelecionadas2019Sim.loc[(df_censoSelecionadas2019Sim["regiao"]== 'Sudeste')]
df1=df_cidade[['nome_capital','idade']]
df2=df1.groupby(['idade','nome_capital']).size().reset_index(name='Total Entrevistados')
fig3 = px.line(df2, x="idade", y="Total Entrevistados", color='nome_capital',title='Evolução Pratica de atividade Fisica ao Longo da Idade Capital Sudeste')
fig3.show()

In [ ]:
# Agrupa e exibe tipço exercico
df_censoSelecionadas2019SimGroup  = df_censoSelecionadas2019Sim.groupby(['tipo_exercicio'], as_index=False)
pequisaNao1 = Counter(df_censoSelecionadas2019Sim['tipo_exercicio'])
pequisaNao1


In [ ]:
df_censoSelecionadas2019Sim.head()

In [ ]:
#Contagem das opções da coluna grau_escolaridade dos entrevistados
df_escolaridade_praticanteAtividade = Counter(df_censoSelecionadas2019Sim['grau_escolaridade'])
df_escolaridade_praticanteAtividade

In [ ]:
#Contagem das opções da coluna grau_escolaridade dos entrevistados
df_escolaridade_Naopraticante= Counter(df_censoSelecionadas2019Nao['grau_escolaridade'])
df_escolaridade_Naopraticante

In [ ]:
#Identificação dos percentuais de escolaridade dos Entrevistados Não praticam
n_censoSelecionadas2019Nao=sum(df_escolaridade_Naopraticante.values())
for x, y in df_escolaridade_Naopraticante.items():
    a = y/n_censoSelecionadas2019Nao*100
    print(str(x) + ': ' +'\n'+ str(round(a,2)) + '%'+'\n')

In [ ]:
#Identificação dos percentuais de escolaridade dos Entrevistados
n_censoSelecionadas2019=sum(df_escolaridade_praticanteAtividade.values())
for x, y in df_escolaridade_praticanteAtividade.items():
    a = y/n_censoSelecionadas2019*100
    print(str(x) + ': ' +'\n'+ str(round(a,2)) + '%'+'\n')


In [ ]:
#Criação de listas para construção do gráfico de escolaridade
lista_df_escolaridade_praticanteAtividade_labels = []
lista_df_escolaridade_praticanteAtividade_valores = []
for x, y in df_escolaridade_praticanteAtividade.items():
    lista_df_escolaridade_praticanteAtividade_labels.append(x)
    lista_df_escolaridade_praticanteAtividade_valores.append(y)

In [ ]:
#Verificação da lista criada
lista_df_escolaridade_praticanteAtividade_valores

In [ ]:
#Verificação da lista criada
lista_df_escolaridade_praticanteAtividade_labels

In [ ]:
#Plotagem do gráfico de escolaridade dos Entrevistados
plt.style.use('seaborn-pastel')
plt.barh(lista_df_escolaridade_praticanteAtividade_labels, lista_df_escolaridade_praticanteAtividade_valores)
plt.ylabel('Escolaridade')
plt.xlabel('número de Praticantes')
plt.title('Total de Praticantes por Escolaridade', fontsize=18)
plt.show()





# Analise de Escolaridade

In [ ]:
#Plotagem Escolaridade"
import plotly.express as px
df =df_atividadeSelecionadas.loc[df_atividadeSelecionadas['ano']== 2019]
figure = px.histogram(df, x = "grau_escolaridade", color = "pratica_exercicio", title= "Atividade Fisica Escolaridade")
figure.show()

# 5. Criação de Modelos de Machine Learning

In [ ]:
#Verifica se existe valores nulo no Data Frame
df_censoSelecionadas2019FinalTreinamento.isnull().sum()

In [ ]:

df_censoSelecionadas2019FinalTreinamento['pratica_exercicio'].replace(2,0, inplace =True)

In [ ]:
#Exibe os valores do Data Frame
df_censoSelecionadas2019FinalTreinamento.head()

In [ ]:
#Exibe informação do Data Frame
df_censoSelecionadas2019FinalTreinamento.info()

In [ ]:
#trata tipo campo 
df_censoSelecionadas2019FinalTreinamento.ordem    = df_censoSelecionadas2019FinalTreinamento.ordem.astype('int64')
df_censoSelecionadas2019FinalTreinamento.ano  = df_censoSelecionadas2019FinalTreinamento.ano.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.civil  = df_censoSelecionadas2019FinalTreinamento.civil.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.idade  = df_censoSelecionadas2019FinalTreinamento.idade.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.sexo   = df_censoSelecionadas2019FinalTreinamento.sexo.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.grau_escolaridade  = df_censoSelecionadas2019FinalTreinamento.grau_escolaridade.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.peso   = df_censoSelecionadas2019FinalTreinamento.peso.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.altura = df_censoSelecionadas2019FinalTreinamento.altura.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.pratica_exercicio  = df_censoSelecionadas2019FinalTreinamento.pratica_exercicio.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.tipo_exercicio     = df_censoSelecionadas2019FinalTreinamento.tipo_exercicio.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.pratica_exercicio_1_vez_na_semana = df_censoSelecionadas2019FinalTreinamento.pratica_exercicio_1_vez_na_semana.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.frequencia_exercicio = df_censoSelecionadas2019FinalTreinamento.frequencia_exercicio.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.duracao_exercicio = df_censoSelecionadas2019FinalTreinamento.duracao_exercicio.astype('uint8')
df_censoSelecionadas2019FinalTreinamento.cor = df_censoSelecionadas2019FinalTreinamento.cor.astype('uint8')


In [ ]:
#Verificação dos valores da coluna Sexo
df_censoSelecionadas2019FinalTreinamento['sexo'].unique()

In [ ]:
#total Pesquisa
pequisa2019Pra = Counter(df_censoSelecionadas2019FinalTreinamento['pratica_exercicio'])
pequisa2019Pra

In [ ]:
#Verificação dos valores da coluna Grau Escolaridade
df_censoSelecionadas2019FinalTreinamento['grau_escolaridade'].unique()

In [ ]:
#Verificação dos valores da coluna Estado Civil
df_censoSelecionadas2019FinalTreinamento['civil'].unique()

In [ ]:
#Verificação dos valores da coluna Pratica de Exercicio
df_censoSelecionadas2019FinalTreinamento['pratica_exercicio'].unique()

In [ ]:
#Primeiras linhas do dataframe df_censoSelecionadas2019Final
df_censoSelecionadas2019FinalTreinamento.head()

In [ ]:
# Exibe total data frame
df_censoSelecionadas2019FinalTreinamento['pratica_exercicio'].value_counts()

In [ ]:
#Importação da função resample da biblioteca sklearn
from sklearn.utils import resample

In [ ]:
#Criação de um dataframe com entrevistados que não praticam atividade fisica
df_censoSelecionadas2019FinalTreinamento_majority = df_censoSelecionadas2019FinalTreinamento[df_censoSelecionadas2019FinalTreinamento.pratica_exercicio==0]
df_censoSelecionadas2019FinalTreinamento_majority.info()

In [ ]:
#Criação de um dataframe com entrevistados que praticam atividade fisica
df_censoSelecionadas2019FinalTreinamento_minority = df_censoSelecionadas2019FinalTreinamento[df_censoSelecionadas2019FinalTreinamento.pratica_exercicio==1]
df_censoSelecionadas2019FinalTreinamento_minority.info()

In [ ]:
#Ajuste no número de entradas do dataframe entrevistados que praticam ativididade fisica
df_censoSelecionadas2019FinalTreinamento_minority_upsampled = resample(df_censoSelecionadas2019FinalTreinamento_minority, 
                                                 replace=True, n_samples=22283, 
                                                 random_state=123)

In [ ]:
#Informações do dataframe ajustado
df_censoSelecionadas2019FinalTreinamento_minority_upsampled.info()

In [ ]:
#Concatenação dos dataframes de entrevistados que praticam atividade fisica e que não praticam 
df_censoSelecionadas2019FinalTreinamento_upsampled = pd.concat([df_censoSelecionadas2019FinalTreinamento_majority, 
                                          df_censoSelecionadas2019FinalTreinamento_minority_upsampled])

In [ ]:
#Informações do dataframe ajustado
df_censoSelecionadas2019FinalTreinamento_upsampled.info()

In [ ]:
#Contagem dos valores de entrevistados que praticam atividade fisica e que não praticam 
df_censoSelecionadas2019FinalTreinamento_upsampled['pratica_exercicio'].value_counts()

In [ ]:
#Importação da função train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
#Divisão para as bases de treinamento
X_train = df_censoSelecionadas2019FinalTreinamento_upsampled.drop(['pratica_exercicio'], axis = 1)
y_train = df_censoSelecionadas2019FinalTreinamento_upsampled.pratica_exercicio

In [ ]:
#Informações do dataframe com os dados para treinamento
X_train.info()

In [ ]:
#Tipo da serie y_train
type(y_train)

In [ ]:
#Criação das bases de teste e treinamento
xtreinamento, xteste, ytreinamento, yteste = train_test_split(X_train, y_train, random_state =0)

In [ ]:
#Informação do dataframe de treinamento
xtreinamento.info()

In [ ]:
#Informação do dataframe de teste
xteste.info()

In [ ]:
#Contagem dos resultados para treinamento
ytreinamento.count()

In [ ]:
#Contagem dos resultados para teste
yteste.count()

In [ ]:
#Criação do modelo utilizando a Árvore de decisão
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
df_censoSelecionadas2019FinalTreinamento_tree = DecisionTreeClassifier(random_state=0)
df_censoSelecionadas2019FinalTreinamento_tree = df_censoSelecionadas2019FinalTreinamento_tree.fit(xtreinamento, ytreinamento)
print("Acurácia: ", df_censoSelecionadas2019FinalTreinamento_tree.score(xtreinamento, ytreinamento))
Train_predict = df_censoSelecionadas2019FinalTreinamento_tree.predict(xteste)
print("Acurácia de previsão: ", accuracy_score(yteste, Train_predict))
print(classification_report(yteste, Train_predict))

In [ ]:
# criando modelo arvore
classificador_arvore = DecisionTreeClassifier(random_state = 1) 
classificador_arvore.fit(xtreinamento,ytreinamento)

In [ ]:
predicao_arvore = classificador_arvore.predict(xteste)
print(predicao_arvore)

In [ ]:
#Gerando matrix de confusão
confusao_nb = confusion_matrix(yteste, Train_predict)
print(confusao_nb)
cmd_arvore = ConfusionMatrixDisplay(confusao_nb, display_labels=classificador_arvore.classes_).plot()

In [ ]:
#Criação do modelo utilizando a Regressão Logística
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr = lr.fit(xtreinamento, ytreinamento)
print("Acurácia: ", lr.score(xtreinamento, ytreinamento))
tp_lr = lr.predict(xteste)
print("Acurácia de previsão: ", accuracy_score(yteste, tp_lr))
print(classification_report(yteste, tp_lr))

In [ ]:
#Validação cruzada para o modelo Regressão Logística
from sklearn.model_selection import cross_val_score
validacao_arvore = cross_val_score(lr, X_train, y_train, scoring='accuracy', cv=5)
print(validacao_arvore.mean())

In [ ]:
#Criação do modelo utilizando Naïve Bayes
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb = nb.fit(xtreinamento, ytreinamento)
print("Acurácia: ", nb.score(xtreinamento, ytreinamento))
tp_nb = nb.predict(xteste)
print("Acurácia de previsão: ", accuracy_score(yteste, tp_nb))
print(classification_report(yteste, tp_nb))

In [ ]:
#Validação cruzada para o modelo Naive bayes
from sklearn.model_selection import cross_val_score
validacao_arvore = cross_val_score(nb, X_train, y_train, scoring='accuracy', cv=5)
print(validacao_arvore.mean())


In [ ]:
#Criação do modelo utilizando Gradiente Descendente
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd = sgd.fit(xtreinamento, ytreinamento)
print("Acurácia: ", sgd.score(xtreinamento, ytreinamento))
tp_sgd = sgd.predict(xteste)
print("Acurácia de previsão: ", accuracy_score(yteste, tp_sgd))
print(classification_report(yteste, tp_sgd))

In [ ]:
#Criação do modelo utilizando KNN (K - Nearest Neighbors)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn = knn.fit(xtreinamento, ytreinamento)
print("Acurácia: ", knn.score(xtreinamento, ytreinamento))
tp_knn = knn.predict(xteste)
print("Acurácia de previsão: ", accuracy_score(yteste, tp_knn))
print(classification_report(yteste, tp_knn))

In [ ]:
#Faz a validação cruzada com 5 folds e ao final exibe a média da acurácia
validacao_knn = cross_val_score(knn, X_train, y_train, scoring='accuracy', cv=5)
print(validacao_knn.mean())

In [ ]:
#Criação do modelo utilizando Randon Forest 
from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier()
rfm = rfm.fit(xtreinamento, ytreinamento)
print("Acurácia: ", rfm.score(xtreinamento, ytreinamento))
tp_rfm = rfm.predict(xteste)
print("Acurácia de previsão: ", accuracy_score(yteste, tp_rfm))
print(classification_report(yteste, tp_rfm))

In [ ]:
#Validação cruzada do classificador Random Forest
validacao_rf = cross_val_score(rfm, X_train, y_train, scoring='accuracy', cv=5)
print(validacao_rf.mean())
